In [30]:
# Importando librerias y paquetes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [16]:
path = './data'

In [17]:
solar = pd.read_csv(path + '/solar.csv')
lunar = pd.read_csv(path + '/lunar.csv')

In [18]:
solar.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Gamma,Eclipse Magnitude,Latitude,Longitude,Sun Altitude,Sun Azimuth,Path Width (km),Central Duration
0,1,-1999 June 12,03:14:51,46438,-49456,5,T,-0.2701,1.0733,6.0N,33.3W,74,344,247,06m37s
1,2,-1999 December 5,23:45:23,46426,-49450,10,A,-0.2317,0.9382,32.9S,10.8E,76,21,236,06m44s
2,3,-1998 June 1,18:09:16,46415,-49444,15,T,0.4994,1.0284,46.2N,83.4E,60,151,111,02m15s
3,4,-1998 November 25,05:57:03,46403,-49438,20,A,-0.9045,0.9806,67.8S,143.8W,25,74,162,01m14s
4,5,-1997 April 22,13:19:56,46393,-49433,-13,P,-1.4670,0.1611,60.6S,106.4W,0,281,NaN,NaN


In [19]:
lunar.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Quincena Solar Eclipse,Gamma,Penumbral Magnitude,Umbral Magnitude,Latitude,Longitude,Penumbral Eclipse Duration (m),Partial Eclipse Duration (m),Total Eclipse Duration (m)
0,1,-1999 June 26,14:13:28,46437,-49456,17,N,t-,-1.0981,0.8791,-0.1922,24S,22W,268.8,-,-
1,2,-1999 November 21,20:23:49,46427,-49451,-16,N,-a,-1.1155,0.8143,-0.1921,15N,98W,233.4,-,-
2,3,-1998 May 17,05:47:36,46416,-49445,-11,P,-t,0.8988,1.2105,0.2069,13S,89E,281.7,102.7,-
3,4,-1998 November 11,05:15:58,46404,-49439,-6,P,-a,-0.4644,2.0382,0.9740,12N,113E,343.4,200.8,-
4,5,-1997 May 6,18:57:01,46392,-49433,-1,T+,pp,0.1003,2.6513,1.6963,11S,92W,322.8,213.5,98.2


In [20]:
# Diccionario para convertir un mes a número
dic_months = {
    'January':1,
    'February': 2,
    'March':3,
    'April':4,
    'May':5,
    'June': 6,
    'July':7,
    'August':8,
    'September':9,
    'October':10,
    'November':11,
    'December':12
}

In [21]:
"""
    Transforma una fecha del formato 1999 June 12 al formato YYYY:MM:DD
"""
def transform_data_month(string_date):
    date_split = string_date.split()
    
    month_number = dic_months[date_split[1]]
    
    return date_split[0]+':'+str(month_number)+':'+date_split[2]

In [22]:
"""
    Transforma una fecha del formato YYYY:MM:DD y una hora en el formato hh:mm:ss a YYYY:MM:DD:hh:mm:ss
"""
def date_time(string_date, string_time):
    return string_date + ':' + string_time

In [23]:
"""
    Transforma una fecha del formato YYYY:MM:DD:hh:mm:ss a una lista de la forma [YYYY, MM, DD, hh, mm, ss]
"""
def datetime_to_list(string_date_time):    
    date_list = string_date_time.split(':')
    
    date_list = [int(i) for i in date_list]

    return date_list

In [24]:
"""
    Tranforma un entero entre 0 y 99 en un string de 2 digitos
"""
def mak_2_dig(x):
    return "{:02d}".format(x)

In [25]:
"""
    Tranforma una Datetime en calendario Juliano o Gregoriano al número de días en calendario Juliano
"""
def date_to_jd(datetime):
    # Se obtiene una listada de la forma [YYYY, MM, DD, hh, mm, ss]
    date_list = datetime_to_list(datetime)
    
    # Se destructuran los valores del datetime
    y, mo, d, h, mn, s = date_list
    
    # Se valida si la fecha está en calendario Gregoriano
    if date_list[:3] >= [1582,10,15]:
        # Formula para el calculo de días en el calendario Juliano de una fecha en calendario Gregoriano
        return 367*y - (7*(y+int((mo+9)/12)))//4 - (3*(int((y+(mo-9)/7)/100)+1))//4+(275*mo)//9+d+1721028.5+h/24+mn/(24*60)+s/86400
    
    # Se tiene en cuenta los 10 días del desfase entre los calendarios
    elif date_list[:3] <= [1582,10,4]:
        # Formula para el calculo de días en el calendario Juliano de una fecha
        return 367*y - (7*(y+5001+int((mo-9)/7)))//4+(275*mo)//9+d+1729776.5+h/24+mn/(24*60)+s/86400

In [26]:
"""
    Transforma el número de días en calendario Juliano a una fecha
"""
def jd_to_date(jd):
    Z = int(jd+0.5)
    F = (jd+0.5)%1
    if Z < 2299161:
        A = Z
    else:
        g = int((Z - 1867216.25) / 36524.25)
        A = Z + 1 + g - g//4 

    B = A + 1524
    C = int((B-122.1) / 365.25)
    D = int(365.25 * C)
    E = int((B-D) / 30.6001)
    
    # Cálculo del día
    d = B - D - int(30.6001*E) + F
    
    # Cálculo del mes
    if E<14:
        mo = E-1
    else:
        mo = E-13    
    
    # Cálculo del año
    if mo >2:
        y = C- 4716
    else:
        y = C - 4715
    
    return str(y)+'-'+mak_2_dig(mo)+'-'+mak_2_dig(int(d))

In [27]:
# Transformamos las fechas
solar['Date Transformed'] = solar['Calendar Date'].apply(lambda x:transform_data_month(x))

#Concatenamos la fecha y la hora
solar['Datetime'] = solar.loc[:, ['Date Transformed', 'Eclipse Time']].apply(lambda x: date_time(x[0], x[1]), axis = 1)

#Calculamos la cantidad de días en calendario  Juliano
solar['Time JD'] = solar['Datetime'].apply(lambda x:date_to_jd(x))

In [28]:
solar.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Gamma,Eclipse Magnitude,Latitude,Longitude,Sun Altitude,Sun Azimuth,Path Width (km),Central Duration,Date Transformed,Datetime,Time JD
0,1,-1999 June 12,03:14:51,46438,-49456,5,T,-0.2701,1.0733,6.0N,33.3W,74,344,247,06m37s,-1999:6:12,-1999:6:12:03:14:51,991085.635313
1,2,-1999 December 5,23:45:23,46426,-49450,10,A,-0.2317,0.9382,32.9S,10.8E,76,21,236,06m44s,-1999:12:5,-1999:12:5:23:45:23,991262.489850
2,3,-1998 June 1,18:09:16,46415,-49444,15,T,0.4994,1.0284,46.2N,83.4E,60,151,111,02m15s,-1998:6:1,-1998:6:1:18:09:16,991440.256435
3,4,-1998 November 25,05:57:03,46403,-49438,20,A,-0.9045,0.9806,67.8S,143.8W,25,74,162,01m14s,-1998:11:25,-1998:11:25:05:57:03,991616.747951
4,5,-1997 April 22,13:19:56,46393,-49433,-13,P,-1.4670,0.1611,60.6S,106.4W,0,281,NaN,NaN,-1997:4:22,-1997:4:22:13:19:56,991765.055509


In [29]:
def get_measures(data, column,print_results=False):
    average = data[column].mean()
    median = data[column].median()
    standard_deviation = data[column].std()
    min_value = data[column].min()
    max_value = data[column].max()
    skewness = data[column].skew(axis = 0)
    kurt = data[column].kurt(axis = 0)
    if print_results:
        print(f"Valor mínimo: {min_value}")
        print(f"Promedio: {average}")
        print(f"Mediana: {average}")
        print(f"Valor máximo: {max_value}")
        print(f"Desviación estandar: {average}")
        print(f"Asimetría: {skewness}")
        print(f"Curtosis: {kurt}")
    return average, median, standard_deviation

def plot_blox_plot(data, column):
    fig = px.box(data, y=column)
    fig.show()

def plot_histogram(data, column):
    fig = px.histogram(data, x=column)
    fig.show()

In [ ]:
get_measures(solar_eclipses, "Delta T (s)", print_results=True)
plot_blox_plot(solar_eclipses, "Delta T (s)")
plot_histogram(solar_eclipses, "Delta T (s)")